In [ ]:
## Load any changes to local modules
%load_ext autoreload
%autoreload 2

import os
import sys

pwd = %pwd
project_dir = '{0}/../../../'.format(pwd)
module_path = os.path.abspath(os.path.join(project_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import re
import numpy as np
%matplotlib inline
from IPython.display import display

import sqlalchemy
import sqlite3

from nbcpact import AnalyzeQuantCompare,Peptide,PeptideGroup,UcbreUtils,PeptidesFromPeptideListBuilder


In [ ]:
file_path = "../../../test/data/peptideList.csv"
peptide_generator = PeptidesFromPeptideListBuilder(peptide_list_file=file_path)
peptides = peptide_generator.generate_peptides()
pep = peptides[0]

print("ip2_peptide={0}, sequence={1}, mod_locs={2}, ptm_indices={3}, area_ratio={4}".format(pep.ip2_peptide, pep.sequence, pep.mod_locs, pep.ptm_indices, pep.area_ratio))
print("area_ratios={0}, annotation={1}, uniprot_ids={2}, run_counter={3}".format(pep.area_ratios, pep.annotation, pep.uniprot_ids, pep.run_counter))
print("decoy={0},unique1={1}".format(pep.decoy, pep.unique1))


"""
ip2_peptide=None, sequence=None, mod_locs=None, ptm_indices=None, area_ratio=None,
                 area_ratios=None, annotation=None, uniprot_ids=None, run_counter=None, decoy=None,
                 unique1=None
"""

In [ ]:
file_path = "../../../../../../projects/nb-cpact/KEA_EN80/Analysis/PD2.1/KEA_isoTOP_DN_High_Med_All.xlsx"
pepgroup_df = pd.read_excel(file_path)

In [ ]:
file_path = "../../../../../../projects/nb-cpact/KEA_EN80/Analysis/PD2.1/KEA_isoTOP_DN_All.pdResult"
pd_connection = sqlite3.connect(file_path)

sqlStr = """
SELECT
Sequence,
ModifiedSequence,
Modifications,
ParentProteinAccessions,
ParentProteinDescriptions,
SpectrumFileName,
QuanChannel,
QuanValueIsoTOPLight,
QuanValueIsoTOPHeavy
FROM TargetPsms
WHERE (QuanChannelID = 1 OR QuanChannelID = 2)
"""
targetPsmsDF = pd.read_sql(sqlStr, pd_connection)

exp_pattern = r'(KEA_EN80\d)_\d.raw'
targetPsmsDF['EXPERIMENT'] = targetPsmsDF.SpectrumFileName.str.extract(exp_pattern)

cys_mod_pattern = re.compile(r'C(\d+)\(isoTO\w*\)')
def __process_modifications(modifications):
    return re.findall(cys_mod_pattern, modifications)

targetPsmsDF['mod_locs'] = targetPsmsDF.Modifications.apply(__process_modifications)


In [ ]:
iso_top_mod_pattern = re.compile(r'.*\dxisoTOP TEV [heavyLight]+ \[(.+)\].*')
cys_mod_pattern = re.compile(r'C(\d+)')

def __process_modifications(modifications):
    match = iso_top_mod_pattern.match(modifications)
    if match:
        mod_locs = list(map(int, re.findall(cys_mod_pattern, match.group(1))))
        return mod_locs
    
def __create_run_counter(row, targetPsmsDF=None, experiments=['KEA_EN801', 'KEA_EN802', 'KEA_EN803']):
    sequence = row['sequence']
    mod_locs = row['mod_locs']
    
    print("File:")
    print(sequence)
    print(mod_locs)

    psms = targetPsmsDF[(targetPsmsDF.Sequence == sequence) & (targetPsmsDF.mod_locs.values == mod_locs)]
    psm_exps = psms['EXPERIMENT']
    run_counter = np.isin(experiments, psm_exps)
    
    print("psms")
    print(psms.Sequence)
    print(psms.mod_locs)
    
    return tuple(run_counter)
    
    
data = {}

## Clean up dataframe
pepgroup_df = pepgroup_df[~pepgroup_df['Abundance Ratio (log2): (IsoTOP Light) / (IsoTOP Heavy)'].isnull()]

## Transfer some values directly
data['uniprot_ids'] = pepgroup_df['Master Protein Accessions']
data['area_ratio'] = pepgroup_df['Abundance Ratio (log2): (IsoTOP Light) / (IsoTOP Heavy)']
data['sequence'] = pepgroup_df['Sequence']

## Process Mods
data['mod_locs'] = pepgroup_df.Modifications.apply(__process_modifications)


# Do later
## Global PTM
data['ptm_indices'] = None
## Detailed protein annot
data['annotation'] = None
## The IP2 semicolon array that RunCounter is made from
data['unique1'] = None


df = pd.DataFrame(data=data)

## Create run_counter array
#data['run_counter'] = pepgroup_df.apply(__create_run_counter, axis=1, targetPsmsDF=targetPsmsDF)
display(df.head(1).apply(__create_run_counter, axis=1, targetPsmsDF=targetPsmsDF))






In [ ]:
np.array_equal([3], [3])

In [ ]:
#targetPsmsDF.mod_locs

def array_equal(mod_locs, a2):
    return np.array_equal(mod_locs, a2)
    
display(targetPsmsDF[targetPsmsDF.mod_locs.apply(array_equal, a2=[3])])

display(targetPsmsDF.index.size)

len(targetPsmsDF.mod_locs.apply(array_equal, a2=[3]))
